In [1]:
import pystan
import numpy as np

In [46]:
stan_model = """
functions {
    matrix rotation_matrix(real angle, int n, int i, int j) {
        matrix[n, n] R = diag_matrix(rep_vector(1, n));
        
        R[i, i] = cos(angle);
        R[i, j] = -sin(angle);
        R[j, i] = sin(angle);
        R[j, j] = cos(angle);
        
        return R;
    }
    
    matrix d_rotation_matrix(real angle, int n, int i, int j) {
        matrix[n, n] dR = diag_matrix(rep_vector(1, n));
        
        dR[i, i] = -sin(angle);
        dR[i, j] = -cos(angle);
        dR[j, i] = cos(angle);
        dR[j, j] = -sin(angle);
        
        return dR;
    }
    
    matrix[] generate_forward_pgivens(vector angles, int n, int p) {
        int d = n*p - p*(p+1)/2;
        int idx;
        matrix[n, n] G;
        matrix[n, n] partial_givens[d+1];
        matrix[n, n] R;
        
        G = diag_matrix(rep_vector(1, n));
        idx = 1;
        partial_givens[1] = G;
        for(i in 1:p){
            for(j in i+1:n){
                R = rotation_matrix(angles[idx], n, i, j);
                G = G * R;
                partial_givens[idx + 1] = G;
                idx = idx + 1;
            }
        }
        
        return partial_givens;      
    }
    
    matrix[] generate_reverse_pgivens(vector angles, int n, int p) {
        int d = n*p - p*(p+1)/2;
        int idx;
        matrix[n, n] G_eye;
        matrix[n, p] G;
        matrix[n, p] partial_givens[d+1];
        matrix[n, n] R;

        G_eye = diag_matrix(rep_vector(1, n));
        G = G_eye[,1:p];
        
        partial_givens[d+1] = G;
        idx = d;
        for(i in 1:p){
            int i_st = p - i + 1;
            for(j in i_st+1:n){
                R = rotation_matrix(angles[idx], n, i_st, n - j + i_st + 1);
                G = R * G;
                partial_givens[idx] = G;
                idx = idx - 1;
            }
        }
        
        return partial_givens;      
    }
    
    matrix[] generate_givens_jacobians(matrix[] partial_givens_forward, matrix[] partial_givens_reverse, vector angles, int n, int p) {
        int d = n*p - p*(p+1)/2;
        matrix[n,p] derivative_list[d];
        matrix[n,d] givens_jacobian[p];
        int idx = 1;
        
        for(i in 1:p){
            for(j in i+1:n){
                matrix[n,n] dR = d_rotation_matrix(angles[idx], n, i, j);
                matrix[n,n] a = partial_givens_forward[idx];
                matrix[n,p] b = partial_givens_reverse[idx + 1];
                
                derivative_list[idx] = a * dR * b;
                idx = idx + 1;
            }
        }
        
        for(i in 1:p) {
            for(j in 1:d) {
                vector[n] t = derivative_list[j][,i];
                matrix[n,d] z = givens_jacobian[i];
                z[,j] = t;
                givens_jacobian[i] = z;
            }
        }
        
        return givens_jacobian;
        
    }
    
    real area_form(vector angles, int n, int p) {
        int d = n*p - p*(p+1)/2;
        int idx;
        matrix[n, n] givens;
        matrix[n, n] partial_givens_forward[d+1];
        matrix[n, p] partial_givens_reverse[d+1];
        matrix[n, d] givens_jacobians[p];
        matrix[d, d] area_mat;
        /**
         * Create Partial Givens
        */
        
        partial_givens_forward = generate_forward_pgivens(angles, n, p);
        partial_givens_reverse = generate_reverse_pgivens(angles, n, p);
        givens = partial_givens_forward[d+1];
        
        givens_jacobians = generate_givens_jacobians(partial_givens_forward, partial_givens_reverse, angles, n, p);
        
        idx = 1;
        for(i in 1:p){
            matrix[d, n-i] one_forms;
            one_forms = (givens'[i+1:n,] * givens_jacobians[i])';
            for(j in 1:n-i) {
                area_mat[,idx] = one_forms[,j]; 
                idx = idx + 1;
            }
        }
        
        return log(determinant(area_mat));
    }
}

data {
    int n;
    int p;
    int d;
    vector[d] angles;
}

parameters {
}

transformed parameters {
    print(area_form(angles, n, p));
}

model {
}
"""

In [47]:
n = 12
p = 11
d = int(n*p - p*(p+1)/2)
data = {'n' : n, 'p' : p, 'd' : d, 'angles' : [np.pi/4 for _ in range(d)]}

In [48]:
fit = pystan.stan(model_code = stan_model, data = data, iter = 1, chains = 1, algorithm = "Fixed_param")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_5c339b1239a657b5e47a016eb06424ce NOW.
